In [127]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyzbar.pyzbar as pyzbar
import copy
import math
import imutils
# --------------- pre-processing --------------------
def pre_processing(image):
    h, w = image.shape[:2]
    #image = cv2.resize(image, (int(w/2), int(h/2)))
    """for i in range(0, h):
        for j in range(0, w):
            if image[i][j][0] <= 30 and image[i][j][1] <= 30 and image[i][j][2] <= 30:
                image[i][j] = 0,0,0
            elif image[i][j][0] >= 230 and image[i][j][1] >= 230 and image[i][j][2] >= 230:
                image[i][j] = 255,255,255"""
    return image

# -------------- find viga --------------------------
def find_viga(data):
    viga = [[], []]
    for d in data:
        if 'viga inicio' in str(d[0]):
            viga[0] = d[2]
        elif 'viga fim' in str(d[0]):
            viga[1] = d[2]
    return viga

def find_reta(data):
    viga = [[], []]
    for d in data:
        if 'viga inicio' in str(d[0]):
            viga[0] = d[3]
        elif 'viga fim' in str(d[0]):
            viga[1] = d[3]
    return viga
            

# -------------- decode image -----------------------
def decode(image):
    qr_data = pyzbar.decode(image)
    return qr_data

def distance(a, b):
    return ((a[0]-b[0])**2+((b[1]-a[1])**2))**1/2

def process_image(viga, image):
    if viga[0][1] != viga[1][1]: #verifico de a altura dos qr codes é a mesma
        angle = np.sin(distance(viga[0][0], viga[0][1]))
        image = imutils.rotate(image, -angle)
    return image
    
def decode_all(image):
    results = decode(image)
    results_a = []
    for  r in results:
        if not 'viga inicio' in str(r[0]) and not 'viga fim' in str(r[0]):
            results_a.append(r)
    return results_a
    
        
    

# --------------- classify --------------------------
def classify(data):
    pontual = []
    distribuida = []
    triangular = []
    apoio1 = []
    apoio2 = []
    for d in data:
        if 'pontual' in str(d[0]):
            pontual.append(d)
        elif 'distr' in str(d[0]):
            distribuida.append(d)
        elif 'triang' in str(d[0]):
            triangular.append(d)
        elif 'apoio 1' in str(d[0]):
            apoio1.append(d)
        elif 'apoio 2' in str(d[0]):
            apoio2.append(d)
    return apoio1, apoio2, pontual, distribuida, triangular
        

    
# ---------------- interpret ----------------------------
def interpret_viga(viga):
    return [viga[0][0]+viga[0][2], viga[1][0]]
    
def interpret_distribuida(distribuida: list):
    forcas_distribuidas = []
    inicio = 0
    fim = 0
    if len(distribuida):
        if distribuida[0][2][0] > distribuida[1][2][0]:
            fim = distribuida[0][2][0]+distribuida[0][2][2]
            inicio = distribuida[1][2][0]
        else:
            fim = distribuida[1][2][0]+distribuida[1][2][2]
            inicio = distribuida[0][2][0]
        weight = str(distribuida[0][0]).replace("\'", "").split('w: ')
        forcas_distribuidas.append([int(weight[1]), inicio, fim])
    return forcas_distribuidas
        


def interpret_pontual(pontual):
    forcas_pontuais = []
    for p in pontual:
        weight = str(distribuida[0][0]).replace("\'", "").split('w: ')
        forcas_pontuais.append([int(weight[1]), p[2][0], p[2][0]+p[2][2]])
        
    return forcas_pontuais


def interpret_triangular(triangular):
    triangular_10 = []
    triangular_6 = []
    triangular_3 = []
    forcas_triangulares = []
    for t in triangular:
        if 'w: 10' in str(t[0]):
            triangular_10.append(t[2])
        elif 'w: 6' in str(t[0]):
            triangular_6.append(t[2])
        elif 'w: 3' in str(t[0]):
            triangular_3.append(t[2])
    
    if len(triangular_10):
        mais_distante = 0
        mais_proximo = 0
        outro = 0
        width = 0
        mais_distante = max(triangular_10[0][0], triangular_10[1][0], triangular_10[2][0])
        mais_proximo = min(triangular_10[0][0], triangular_10[1][0], triangular_10[2][0])
        for t in triangular_10:
            if t[0] != mais_distante and t[0] != mais_proximo:
                outro = t[0]
            if t[0] == mais_distante:
                width = t[2]
        if abs(outro-mais_proximo) >= 50:
            forcas_triangulares.append([10, mais_proximo, mais_distante+width])
        else:
            forcas_triangulares.append([10, mais_distante+width, mais_proximo])
            
    if len(triangular_6):
        mais_distante = 0
        mais_proximo = 0
        outro = 0
        width = 0
        mais_distante = max(triangular_6[0][0], triangular_6[1][0], triangular_6[2][0])
        mais_proximo = min(triangular_6[0][0], triangular_6[1][0], triangular_6[2][0])
        for t in triangular_6:
            if t[0] != mais_distante and t[0] != mais_proximo:
                outro = t[0]
            if t[0] == mais_distante:
                width = t[2]
        if abs(outro-mais_proximo) >= 10:
            forcas_triangulares.append([6, mais_proximo, mais_distante+width])
        else:
            forcas_triangulares.append([6, mais_distante+width, mais_proximo])
    
    if len(triangular_3):
        mais_distante = 0
        mais_proximo = 0
        outro = 0
        width = 0
        mais_distante = max(triangular_3[0][0], triangular_3[1][0], triangular_3[2][0])
        mais_proximo = min(triangular_3[0][0], triangular_3[1][0], triangular_3[2][0])
        for t in triangular_3:
            if t[0] != mais_distante and t[0] != mais_proximo:
                outro = t[0]
            if t[0] == mais_distante:
                width = t[2]
        if abs(outro-mais_proximo) >= 10:
            forcas_triangulares.append([3, mais_proximo, mais_distante+width])
        else:
            forcas_triangulares.append([3, mais_distante+width, mais_proximo])
            
            
    return forcas_triangulares

def interpret_apoios(apoios):
    apoio_1 = []
    apoio_2 = []
    for a in apoios:
        if 'apoio 1' in str(a[0]):
            apoio_1.append([a[2][0], a[2][0]+a[2][2]])
        elif 'apoio 2' in str(a[0]):
            apoio_2.append([a[2][0], a[2][0]+a[2][2]])
    return apoio_1, apoio_2

    


        
        
                
        
                
        
    
# --------------- main ------------------------------
image = cv2.imread('img/distribuida_10_new_torto.png')[:,:,::-1]
image = pre_processing(image)
data = decode(image)
viga = find_viga(data)
viga_reta = find_reta(data)
image = process_image(viga_reta, image)
all_data = decode_all(image)
apoio1, apoio2, pontual, distribuida, triangular = classify(all_data)

interpret_viga(viga)
interpret_apoios(apoio1+apoio2)
interpret_triangular(triangular)
interpret_pontual(pontual)
interpret_distribuida(distribuida)



[[10, 382, 846]]